In [1]:
# main.py
import logging
from tkinter import filedialog, Text, messagebox

# Import custom modules
from importable import FileReader, Serialization, Deserialization
from processes import FileProcessor
from exportable import Exportable
from GUImaker import *

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def main():
    """Main function to run the GUI and process files."""
    # Initialize and run the GUI
    app = GUIMaker()
    app.start()

if __name__ == "__main__":
    main()

Error: You need to submit your path first and check if it's correct.
Error: You need to submit your path first and check if it's correct.
All entries have been deleted. Please resubmit both new data paths and a new workspace.


2024-10-07 15:36:04.070 Python[44522:2104851] +[CATransaction synchronize] called within transaction
2024-10-07 15:36:08.561 Python[44522:2104851] +[CATransaction synchronize] called within transaction


Error: No file path provided for the report.


In [2]:
#Guimaker.py
import os
import time
import logging
import tkinter as tk
import pandas as pd  
import io
import sys

from tkinter import filedialog
import json
from importable import Serialization, Deserialization
from processes import FileProcessor
from exportable import Exportable

# Global dictionary to store confirmed paths
confirmed_paths = {"datasources": [], "workspace": None}


class GUIMaker:
    
    
    def __init__(self):
        self.datasource_entry = None
        self.window = tk.Tk()
        self.window.geometry("800x800")
        self.window.title("ReportCreatorGUI")
        self.window.configure(bg="light grey")  # Set window background color

        # Default save format
        self.save_format = tk.StringVar(value="Pickle")
        
        # Create an instance of Exportable
        self.exportable = Exportable(self)

        # Create widgets including the console output
        self.create_widgets()


    def create_widgets(self):
        
        # Widgets for Data Source
        
        datasource_label = tk.Label(self.window, text="Path data source", bg="light grey")
        datasource_label.grid(row=0, column=0, padx=10, pady=5, sticky="ew")

        self.datasource_entry = tk.Entry(self.window, width=35, font=('Arial', 16), fg='green', bg='beige')
        self.datasource_entry.grid(row=1, column=0, padx=10, pady=5, sticky="ew")

        confirm_datasource_button = tk.Button(self.window, text="Confirm data source",
                                               command=lambda: self.confirm(self.datasource_entry, "datasources"))
        confirm_datasource_button.grid(row=1, column=1, pady=5, sticky="ew")

        delete_button = tk.Button(self.window, text="Delete data paths and workspace",
                                  command=self.delete_entries)
        delete_button.grid(row=3, column=1, pady=5, sticky="ew")

        # Widgets for Workspace
        
        workspace_label = tk.Label(self.window, text="Path to working space", bg="light grey")
        workspace_label.grid(row=3, column=0, padx=10, pady=5, sticky="ew")

        self.workspace_entry = tk.Entry(self.window, width=35, font=('Arial', 16), bg="beige")
        self.workspace_entry.grid(row=4, column=0, padx=10, pady=5, sticky="ew")

        confirm_workspace_button = tk.Button(self.window, text="Confirm path to your working space",
                                              command=lambda: self.confirm(self.workspace_entry, "workspace"))
        confirm_workspace_button.grid(row=4, column=1, pady=5, sticky="ew")

        # Save Button
        
        save_data_to_workspace_button = tk.Button(self.window, width=35, text="Save data to Workspace",
                                                   command=self.save_data_to_workspace)
        save_data_to_workspace_button.grid(row=5, column=0, padx=10, pady=5, sticky="ew")

        # Configure the layout for radio buttons
        
        radio_button_frame = tk.Frame(self.window, bg="light grey")
        radio_button_frame.grid(row=5, column=1, padx=10, pady=5, sticky="ew")
     
        # Use the frame to hold the radio buttons, this will prevent them from overlapping
        pickle_radio = tk.Radiobutton(radio_button_frame, text="Pickle", variable=self.save_format, value="Pickle", bg="light grey")
        pickle_radio.pack(side="left", padx=5, pady=5, fill="x", expand=True)

        json_radio = tk.Radiobutton(radio_button_frame, text="JSON", variable=self.save_format, value="JSON", bg="light grey")
        json_radio.pack(side="left", padx=5, pady=5, fill="x", expand=True)

        other_radio = tk.Radiobutton(radio_button_frame, text="Other", variable=self.save_format, value="Other", bg="light grey")
        other_radio.pack(side="left", padx=5, pady=5, fill="x", expand=True)
        
        # Console Output Text Widget
        self.console_output_1 = tk.Text(self.window, height=10, width=80, bg="black", fg="white", font=("Arial", 12))
        self.console_output_1.grid(row=8, column=0, columnspan=2, padx=10, pady=10, sticky="ew")

        # Second console Text Widget with scrollbar
        console_frame = tk.Frame(self.window)  # Create a frame for the second console and its scrollbar
        console_frame.grid(row=10, column=0, columnspan=1, padx=8, pady=10, sticky="ew")

        self.console_output_2 = tk.Text(console_frame, height=15, width=50, bg="black", fg="white", font=("Arial", 12))
        self.console_output_2.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Create a scrollbar for the second console output
        scrollbar = tk.Scrollbar(console_frame, command=self.console_output_2.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
    
        self.console_output_2['yscrollcommand'] = scrollbar.set  # Link scrollbar with text widget

        clear_console_button = tk.Button(self.window, text="Clear Consoles", command=self.clear_console)
        clear_console_button.grid(row=9, column=1, pady=5, sticky="ew")
 
        # Create a frame for the buttons beside the second console
        button_frame = tk.Frame(self.window, bg="light grey")  # Frame to hold the buttons
        button_frame.grid(row=10, column=1, padx=10, pady=10, sticky="ew")

        # Deserialization Button
        deserialize_button = tk.Button(button_frame, text="Deserialize", command=self.deserialize_file)
        deserialize_button.pack(pady=5, fill=tk.X)

        # Report Creator Button
        report_creator_button = tk.Button(button_frame, text="Create Report", command=self.exportable.create_report)
        report_creator_button.pack(pady=5, fill=tk.X)       
        
        # Configure grid weights for the frame and other widgets
                
        for i in range(30):
            self.window.grid_rowconfigure(i, weight=1)  # Adjust the number based on your rows
        for i in range(3):
            self.window.grid_columnconfigure(i, weight=1)  # Adjust the number based on your columns


    def confirm(self, entry, label):
        """Confirm and store the path entered in the corresponding entry widget."""
        path = entry.get()
        if path:
            try:
                if label == "datasources":
                    if os.path.isfile(path):
                        confirmed_paths["datasources"].append(path)
                        self.log_to_console(f"Added data source path: {path}")
                    else:
                        self.log_to_console(f"Error: Data source does not exist: {path}")
                else:
                    if os.path.isdir(path):
                        confirmed_paths[label] = path
                        self.log_to_console(f"Saved {label} as {path}.")
                    else:
                        self.log_to_console(f"Error: Workspace does not exist: {path}.")
            except Exception as e:
                self.log_to_console(f"Error checking path: {e}")

        else:
            self.log_to_console("Error: You need to submit your path first and check if it's correct.")


    def delete_entries(self):
        """Clear all entries and reset the confirmed paths dictionary."""
        self.datasource_entry.delete(0, tk.END)
        self.workspace_entry.delete(0, tk.END)
        confirmed_paths["datasources"] = []
        confirmed_paths["workspace"] = None
        self.log_to_console("All entries have been deleted. Please resubmit both new data paths and a new workspace.")


    def save_data_to_workspace(self):
        """Save data to the workspace if paths are confirmed and valid."""
        if not confirmed_paths["datasources"] or not confirmed_paths["workspace"]:
            self.log_to_console("Error: You need to confirm both data source paths and workspace path before saving.")
            return

        selected_format = self.save_format.get()
        self.log_to_console(f"Confirmed paths are: {confirmed_paths}")
        self.log_to_console(f"Selected format for saving: {selected_format}")

        # Call FileProcessor to read the data and serialize it
        file_processor = FileProcessor(confirmed_paths, selected_format)

        try:
            result_message = file_processor.process_files()  # Process files to read and serialize
            if result_message:  # Check if result_message is not None
                self.log_to_console(result_message)  # Log the result message
            else:
                self.log_to_console("No result message returned from processing.")
        except Exception as e:
            self.log_to_console(f"Error during file processing: {e}")


    def log_to_console(self, message):
        """Log messages to the console output widget."""
        self.console_output_1.insert(tk.END, message + "\n")
        self.console_output_1.see(tk.END)  # Scroll to the end of the console output
        

        print(message)


    def clear_console(self):
        """Clear all the text in the console output."""
        self.console_output_1.delete(1.0, tk.END)
        self.console_output_2.delete(1.0, tk.END) 


    def deserialize_file(self):
        """Deserialize the selected file and print output to console 2."""
        file_path = self.get_deserialization_file_path()
        if not file_path:
            self.log_to_console_2("Error: No file path provided for deserialization.")
            return

        deserializer = Deserialization()
    
        # Call the deserialize_data function from processes.py
        deserialized_data = deserializer.deserialize_data(file_path)  

        # Optionally log the deserialized data if it's a list or similar
        if isinstance(deserialized_data, str):  # Check if it's an error message
            self.log_to_console_2(deserialized_data)
        else:
            self.log_to_console_2("Deserialized data:")
            self.log_to_console_2(f"{deserialized_data[:100]}")  # Log only the first 1000 characters


    def get_deserialization_file_path(self):
        """Open a file dialog to select a file for deserialization."""
        file_path = tk.filedialog.askopenfilename(
            title="Select a file for deserialization",
            filetypes=[("Pickle files", "*.pkl"), ("JSON files", "*.json"), ("All files", "*.*")]
        )
        return file_path

    def log_to_console_2(self, message):
        """Log messages to the second console output widget."""
        self.console_output_2.insert(tk.END, message + "\n")
        self.console_output_2.see(tk.END)


    def start(self):
        """Start the GUI event loop."""
        self.window.mainloop()


In [3]:
#importable.py
import os
import pandas as pd
import pickle
import xml.etree.ElementTree as ET
import pyreadr
import time
import json

import tkinter as tk
from tkinter import filedialog


'''
Contains the classes FileReader, Serialization and Deserialization. 

FileReader reads in the file depending on its original datastructure so that the datastructure
is not lost under serialization. 

Serialization class serializes the files according to the serialization of choice, as a byte file 
with Python's pickle or as a readable JSON format.

Implemented?|  Filetype   |  Original datastructure |                     Serialized structure 
            |             |                         |        Pickle                      |  JSON
----------------------------------------------------------------------------------------------------------
    YES     |     CSV     | Tables (Rows/Column)    | List of Dictionaries or pandas DF  | Str
----------------------------------------------------------------------------------------------------------
    YES     |    Excel    | Tables (Rows/Column)    | List of Dictionaries or pandas DF  | Dictionary
----------------------------------------------------------------------------------------------------------
    YES     |    JSON     | Dict. or list of Dict.  |       -                            | Kept as original
-----------------------------------------------------------------------------------------------------------
  NOT YET   |    XML      | Tables (Rows/Column)    | List of Dictionaries or pandas DF  | 
-----------------------------------------------------------------------------------------------------------
  NOT YET   |   R/RData   | Various (Vectors, Lists,| List of Dictionaries or pandas DF  |
            |                     DataFrames)       |                                    | 
-----------------------------------------------------------------------------------------------------------
  NOT YET   |     SQL     |   SQL code              |                                    | 
-----------------------------------------------------------------------------------------------------------
 HTTP/HTTPS handling already possible but I need to check the security risks for that. 
 or maybe create a whitelist of acceptable sites. 
 
'''


class FileReader:
    
    
    def read_file(self, file_path: str):
        """Reads a file based on its extension and returns the data in an appropriate format."""
        data = None  # Initialization 
        
        # Getting the file extension
        _, file_extension = os.path.splitext(file_path)
        file_extension = file_extension.lower()

        # Check the file extension and use the appropriate reading method
        try:
            if file_extension == '.csv':
                with open(file_path, 'r') as file:
                    data = file.read()
                print("The file is a .CSV File.")
                
            elif file_extension == '.json':
                with open(file_path, 'r') as file:
                    data = json.load(file)  # Directly load JSON into a dictionary
                print("The file is a .JSON File.")
                         
            elif file_extension in ['.xls', '.xlsx']:
                data = pd.read_excel(file_path)  # Reads it in using pandas anyhow because Excel having limited row numbers
                print("The file is an Excel File (.xls/.xlsx).")
                
            elif file_extension == '.xml':
                tree = ET.parse(file_path)
                root = tree.getroot()
                data = ET.tostring(root, encoding='unicode')
                print("XML File Content as String.")
                            
            elif file_extension in ['.r', '.rdata']:
                result = pyreadr.read_r(file_path)
                data = result  # result is a dictionary-like object with dataframes
                print("R File Content in Dictionary Format.")
                
            else:
                print(f"Error: Unsupported file type: {file_extension}.")
                
        except Exception as e:
            print(f"Error reading file: {e}")

        return data


class Serialization:
    
    
    def serialize(self, data, workspace, format, filename=None, original_filename=None):
        """
        Serialize data to the specified format and save it to the workspace.
        """
        # Check if the workspace path exists
        if not os.path.exists(workspace):
            raise FileNotFoundError("Workspace path does not exist.")

        # Validate the format before proceeding
        if format not in ["Pickle", "JSON"]:
            raise ValueError(f"Unsupported serialization format '{format}'. Please choose 'Pickle' or 'JSON'.")

        if original_filename and format == "JSON":
            _, file_extension = os.path.splitext(original_filename)
            if file_extension.lower() == ".json":
                raise ValueError("Cannot serialize a .json file as .json again.")

        # Generate a filename using the original filename as the base, if provided
        if not filename and original_filename:
            extension = "pkl" if format == "Pickle" else "json"
            base_name = os.path.splitext(os.path.basename(original_filename))[0]
            timestamp = time.strftime("%Y%m%d-%H%M%S")
            filename = f"{base_name}_{timestamp}.{extension}"
        elif not filename:
            raise ValueError("Filename must be provided if original_filename is not given.")

        file_path = os.path.join(workspace, filename)

        try:
            # Serialize based on the selected format
            if format == "Pickle":
                with open(file_path, 'wb') as pickled_file:
                    pickle.dump(data, pickled_file)
                return f"Data successfully saved as Pickle to: {file_path}"

            elif format == "JSON":
                with open(file_path, 'w') as json_file:
                    json.dump(data, json_file, indent=4)
                return f"Data successfully saved as JSON to: {file_path}"

        except Exception as e:
            raise RuntimeError(f"Error serializing data: {e}")
        

class Deserialization:

    
    def __init__(self, os_module=os):
        self.os = os_module  # Store a reference to the os module
        
    
    def deserialize_data(self, file_path):
        """
        Deserialize data from a given file based on its extension.
        """
        # Check if the file exists
        if not self.os.path.isfile(file_path):  # Use self.os instead of os
            return f"Error: The file does not exist: {file_path}"

        # Get the file extension and convert it to lowercase
        _, file_extension = self.os.path.splitext(file_path)  # Use self.os instead of os
        file_extension = file_extension.lower()

        try:
            # Handle Pickle (.pkl) files
            if file_extension == '.pkl':
                with open(file_path, 'rb') as file:
                    data = pickle.load(file)
                    return data  # Return the deserialized data

            # Handle JSON (.json) files
            elif file_extension == '.json':
                with open(file_path, 'r') as file:
                    data = json.load(file)
                    return data  # Return the deserialized data

            # If the file type is not supported
            else:
                return f"Error: Unsupported file type: {file_extension}. Please provide a .pkl or .json file."

        except Exception as e:
            return f"Error during deserialization: {e}"
        

    def deserialize_file(self, output_console_1, output_console_2):
        """Deserialize the selected file and log output to the specified consoles."""
        file_path = self.get_deserialization_file_path()
        if not file_path:
            output_console_1.insert("end", "Error: No file path provided for deserialization.\n")
            return

        # Call the existing deserialize_data method
        deserialized_data = self.deserialize_data(file_path)

        if isinstance(deserialized_data, str):  # Check if it's an error message
            output_console_1.insert("end", deserialized_data)
        else:
            output_console_2.insert("end", "Deserialized data:\n")
            output_console_2.insert("end", f"{deserialized_data[:100]}")  # Log only the first 100 characters
         
         
    def get_deserialization_file_path(self):
        """Open a file dialog to select a file for deserialization."""
        file_path = tk.filedialog.askopenfilename(
            title="Select a file for deserialization",
            filetypes=[("Pickle files", "*.pkl"), ("JSON files", "*.json"), ("All files", "*.*")]
        )
        return file_path
    

In [4]:
#processes.py

import os
import time
import logging

from importable import FileReader, Serialization


'''
File functions: 

Reads in the commands from the GUI (GUImaker.py) and processes with the serialization
by reading in the datafile and serializing (importable.py) it according to the selected format.

Prints messages during the processing to both the log and the GUI console, in order to ease debugging. 
(both error messages and success messages).

'''

import tkinter as tk
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


class FileProcessor:
    
    
    def __init__(self, confirmed_paths, selected_format="Pickle"):
        self.datasources = confirmed_paths.get("datasources", [])
        self.workspace = confirmed_paths.get("workspace")
        self.selected_format = selected_format
        self.file_reader = FileReader()  # Assuming this class exists for reading files
        self.serializer = Serialization()  # Assuming this class exists for serialization
         
    
    def process_files(self):
        """Process each data source and serialize the content."""
        if not self.datasources or not self.workspace:
            logging.error("No paths were provided for data sources or workspace.")
            return "No paths were provided for data sources or workspace."

        result_messages = []  # List to capture all result messages

        for data_source in self.datasources:
            if not os.path.isfile(data_source):
                logging.error(f"Data source does not exist: {data_source}")
                result_messages.append(f"Data source does not exist: {data_source}")
                continue

            logging.info(f"Reading data from: {data_source}")
            data = self.file_reader.read_file(data_source)

            if data is not None:
                try:
                    # Serialize the data and capture the saved file path
                    saved_file_path = self.serializer.serialize(
                        data=data,
                        workspace=self.workspace,
                        format=self.selected_format,
                        original_filename=data_source  # Pass the original filename for better naming
                    )
                    # Log and collect a success message including the saved file path
                    success_message = f"Data successfully saved from: {data_source} to {saved_file_path}"
                    logging.info(success_message)
                    result_messages.append(success_message)
                    
                except Exception as e:
                    error_message = f"Error serializing data from {data_source}: {e}"
                    logging.error(error_message)
                    result_messages.append(error_message)
            else:
                error_message = f"Failed to read data from: {data_source}"
                logging.error(error_message)
                result_messages.append(error_message)

        return "\n".join(result_messages)  # Join all messages for return

In [5]:
#exportable.py

import tkinter as tk
from tkinter import filedialog
import io
import sys

class Exportable:
    
    
    def __init__(self, gui):
        self.gui = gui  # Reference to the GUIMaker instance


    def create_report(self):
        """Open a Python file for editing and running."""
        file_path = self.get_python_file_path()
        if not file_path:
            self.gui.log_to_console("Error: No file path provided for the report.")
            return

        with open(file_path, 'r') as file:
            report_code = file.read()

        # Open a new window for editing the Python script
        editor_window = tk.Toplevel(self.gui.window)
        editor_window.title("Edit Report")
        editor_text = tk.Text(editor_window, height=20, width=80)
        editor_text.insert(tk.END, report_code)
        editor_text.pack()

        run_button = tk.Button(editor_window, text="Run Report", command=lambda: self.run_report(editor_text.get("1.0", tk.END)))
        run_button.pack()


    def get_python_file_path(self):
        """Open a file dialog to select a Python file."""
        file_path = filedialog.askopenfilename(
            title="Select a Python file",
            filetypes=[("Python files", "*.py"), ("All files", "*.*")]
        )
        return file_path


    def run_report(self, report_code):
        """Execute the report code and log messages to console 1 and output to console 2."""
        # Capture output
        output_buffer = io.StringIO()
        sys.stdout = output_buffer  # Redirect stdout to capture print statements
        
        try:
            exec(report_code)  # Run the report code
            output = output_buffer.getvalue()
            if output:
                self.gui.log_to_console_2(output)  # Log output to console 2
            else:
                self.gui.log_to_console_2("No output produced.")
                
        except Exception as e:
            self.gui.log_to_console(f"Error executing report: {e}")  # Log error to console 1
        finally:
            sys.stdout = sys.__stdout__  # Reset stdout
